<a href="https://colab.research.google.com/github/jamesrichardbunting/neurodegeneration_pollution/blob/main/postcode_coordinate_lookup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data wrangling

## 1.1 Combine the postcode datasets

Postcode data is provided as open data by the Office for National Statistics (ONS). 

All 1.7 million UK postcodes are provided, along the administrative area and NHS ward in which they lie, and their 'easting' and 'northing' coordinates. 

Easting and northings correspond to the postcodes x and y coordinates (respectively) on a grid reference map of the UK.

The data are provided as 120 .CSV files and, in order to make later cross-referencing against pollution data easier, these individual files need to be collated into a single, easily searchable file. 


In [1]:
# Import packages
import pandas as pd
import numpy as np
import os
import glob

In [ ]:
# Search the working directory for all .CSV files and save the file names to a new variable  
extension = '.csv'
csv_files = [i for i in glob.glob(f"*{extension}")]

In [ ]:
# Combine the files using the pd.concat() and pd.read_csv() methods, selecting only the postcode, eastings and northings data
comb_postcodes = pd.concat([pd.read_csv(file, header=None, usecols=[0,2,3]) for file in csv_files], ignore_index=True)

In [ ]:
# Check this has been successful
comb_postcodes.head()

,Postcode,x,y
0,OL1 1AA,393482,404988
1,OL1 1AD,392764,404929
2,OL1 1AL,392864,404822
3,OL1 1AN,392560,404725
4,OL1 1AP,392994,404904


In [ ]:
comb_postcodes.shape

(1719485, 3)

In [ ]:
# Add column headers
comb_postcodes.columns = ["Postcode", "x", "y"]

## 1.2 Combine the pollution datasets

Pollution data is provided as open data by the Department for Environment, Food and Rural Affairs (Defra). 

The data I am working with are modelled (ie, predicted) background pollution maps, provided at 1km x 1km resolution where each 1km square is identified with x and y coordinates.

These data are provided as .CSV files - one file per pollutant, per year from 2002 - 2019.

For this first phase of analysis I will be concentrating on PM2.5 and need to collate yearly PM2.5 pollution maps into a time-series dataset, suitable for longitudinal analysis. 

In [2]:
# Search the working directory for all files with a filename using 'map' as a prefix
prefix = 'map*'
pollution_files = [k for k in glob.glob(f"*{prefix}")]

In [18]:
# Combine the files using the pd.concat() and pd.read_csv() methods, selecting only the 
comb_pollution = pd.concat([pd.read_csv(file, skiprows=4, usecols=[3]) for file in pollution_files], ignore_index=True, axis = 1)

In [4]:
#Check that this was successful
comb_pollution.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,pm252019g,pm252003grav,pm2505ac,pm252008g,pm252012g,pm252014g,pm252011g,pm252007g,pm252017g,pm252015g,pm252018g,pm252010g,pm252002,pm252016g,pm252006gh,pm252009g,pm252004g,pm252013g
1,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
2,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
3,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
4,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING


I need to extract the year from the first row, make these the column headers and then organise the columns in chronological order. 

In [19]:
# Firstly, separate the first row from the data, saving to a new variable
yearly_headers = comb_pollution.iloc[:1 , :]
comb_pollution = comb_pollution.iloc[1: , :]

In [20]:
# Convert the first row variable to a list 
yearly_headers = yearly_headers.values.tolist()
yearly_headers

[['pm252019g',
  'pm252003grav',
  'pm2505ac',
  'pm252008g',
  'pm252012g',
  'pm252014g',
  'pm252011g',
  'pm252007g',
  'pm252017g',
  'pm252015g',
  'pm252018g',
  'pm252010g',
  'pm252002',
  'pm252016g',
  'pm252006gh',
  'pm252009g',
  'pm252004g',
  'pm252013g']]

In [21]:
# Because this has been converted from a Dataframe, it is a 'list within a list' so select only the inner list
yearly_headers = yearly_headers[0]
yearly_headers

['pm252019g',
 'pm252003grav',
 'pm2505ac',
 'pm252008g',
 'pm252012g',
 'pm252014g',
 'pm252011g',
 'pm252007g',
 'pm252017g',
 'pm252015g',
 'pm252018g',
 'pm252010g',
 'pm252002',
 'pm252016g',
 'pm252006gh',
 'pm252009g',
 'pm252004g',
 'pm252013g']

In [22]:
# Define a simple function to extract the date from these values
def date_extractor(lst):
    for i in range(len(lst)):
        lst[i] = lst[i][4:8]
    return lst

In [23]:
# Apply the function and check the results
yearly_headers = date_extractor(yearly_headers)
yearly_headers

['2019',
 '2003',
 '05ac',
 '2008',
 '2012',
 '2014',
 '2011',
 '2007',
 '2017',
 '2015',
 '2018',
 '2010',
 '2002',
 '2016',
 '2006',
 '2009',
 '2004',
 '2013']

This has worked but, because of an inconsistency with the way the year was recorded in 2005, (it was abbrevated to '05') I will have to fix this date manually.

In [26]:
# Fix the date format for 2005 and check the result
yearly_headers[2] = '2005'
yearly_headers

['2019',
 '2003',
 '2005',
 '2008',
 '2012',
 '2014',
 '2011',
 '2007',
 '2017',
 '2015',
 '2018',
 '2010',
 '2002',
 '2016',
 '2006',
 '2009',
 '2004',
 '2013']

In [27]:
# Apply these years as header columns to the PM2.5 dataset
comb_pollution.columns = yearly_headers
comb_pollution.head()

,2019,2003,2005,2008,2012,2014,2011,2007,2017,2015,2018,2010,2002,2016,2006,2009,2004,2013
1,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
2,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
3,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
4,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
5,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING


In [28]:
comb_pollution = comb_pollution.reindex(sorted(comb_pollution.columns), axis=1)
comb_pollution.head()

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
2,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
3,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
4,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
5,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
